<a href="https://colab.research.google.com/github/mukundagg/summer-of-code-2023/blob/main/machine-learning/week1/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [141]:
!pip install undetected_chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [142]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [144]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
!ls

In [145]:
import pandas as pd, numpy as np
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import lxml, sklearn, requests

# Note: We used the following datasets from kaggle - laptop-price-dataset
# Objective: Budget estimator for Laptop given features such as brand, OS, weight, screen-size, RAM & processor

In [174]:
laptopData2022 = pd.read_csv("dataset/laptop_data.csv");

In [175]:
#laptopData2022.drop(columns=['Importer', 'Packer', 'Net Quantity', 'ASIN'], inplace=True);
#laptopData2022.drop(columns=['Unnamed: 0'], inplace=True);
laptopData2022.sample(5)

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
531,531,Toshiba,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,500GB HDD,Intel HD Graphics 620,Windows 10,2.0kg,45820.8000
728,728,Dell,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,8GB,1TB HDD,AMD Radeon R7 M445,Linux,2.32kg,31409.6256
292,292,HP,Notebook,17.3,1600x900,Intel Core i3 6006U 2GHz,8GB,1TB HDD,Intel HD Graphics 520,Windows 10,2.54kg,26053.9200
915,915,Lenovo,Notebook,13.3,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 620,Windows 10,1.44kg,46939.6800
390,390,Asus,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,3kg,55357.9200


In [176]:
print(laptopData2022.info())
print(laptopData2022.isnull().sum())
# The good news: No NULL values -- no imputation (ahead, its included for reference), the not so good one -- We need to encode most of the columns and bin some of them

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   TypeName          1303 non-null   object 
 3   Inches            1303 non-null   float64
 4   ScreenResolution  1303 non-null   object 
 5   Cpu               1303 non-null   object 
 6   Ram               1303 non-null   object 
 7   Memory            1303 non-null   object 
 8   Gpu               1303 non-null   object 
 9   OpSys             1303 non-null   object 
 10  Weight            1303 non-null   object 
 11  Price             1303 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 122.3+ KB
None
Unnamed: 0          0
Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory      

In [177]:
laptopData2022['Ram'] = list(map(lambda k : int(k[:-2]), laptopData2022['Ram']))

In [178]:
cpuManufacturerList, cpuType, cpuGHz = [], [], []
for i in laptopData2022['Cpu']:
  x = str(i).strip().split();
  cpuManufacturerList.append(x[0]);
  cpuType.append(x[1] + x[2]);
  cpuGHz.append(float(x[-1][:-3]));

In [179]:
gpuManufacturerList, gpuFamily, gpuModelNo = [], [], []
for i in laptopData2022['Gpu']:
  x = str(i).strip().split();
  gpuManufacturerList.append(x[0]);
  gpuFamily.append(x[1]);
  gpuModelNo.append(x[-1]);

In [180]:
laptopData2022["Cpu_Manufacturer"] = cpuManufacturerList;
laptopData2022['Cpu_Type'] = cpuType;
laptopData2022['Cpu_GHz'] = cpuGHz;
newCpuTypes = [];
for i in laptopData2022['Cpu_Type']:
  if '-' in i:
    i = i[:i.index('-')]
  newCpuTypes.append(i)
laptopData2022['Cpu_Type'] = newCpuTypes
laptopData2022.drop(columns=['Cpu'], inplace=True)

In [181]:
laptopData2022["Gpu_Manufacturer"] = gpuManufacturerList;
#X["Gpu_Family"] = gpuFamily;
#X["Gpu_ModelNo"] = gpuModelNo;
laptopData2022.drop(columns=['Gpu'], inplace=True)
#print(X.head())

In [182]:
laptopData2022['Weight'] = list(map(lambda k : float(k[:-2]), laptopData2022['Weight']))

In [183]:
laptopData2022['Memory_Type'] = list(map(lambda k : k.split()[-1], laptopData2022['Memory']))

In [184]:
laptopData2022['ScreenResolution'] = list(map(lambda k : k.split()[-1].split('x'), laptopData2022['ScreenResolution']))

In [185]:
laptopData2022['ScreenResolution'] = list(map(lambda k: int(k[0]) * int(k[1]), laptopData2022['ScreenResolution']))

In [186]:
memSz = []
for i in laptopData2022['Memory']: #X['Memory_Size']
  x = i.split();
  ans = 0.0;
  for j in x:
    if 'GB' in j: ans += int(j[:j.index('GB')])
    elif 'TB' in j: ans += 1024*float(j[:j.index('TB')]);
    elif 'MB' in j: ans += int(j[:j.index('MB')])/1024.0;
  memSz.append(ans);
laptopData2022['Memory_Size'] = memSz

In [187]:
laptopData2022.drop(columns=['Memory'], inplace=True)
laptopData2022.drop(columns=['ScreenResolution'], inplace=True)
#laptopData2022.drop(columns=['Cpu_GHz'], inplace=True)

In [191]:
#laptopData2022.drop(columns='Unnamed: 0', inplace=True) # -- Uncomment if running again to remove error for newly created index
# Remove outlier data (very rare data with 1 sample only)
for i in X.columns:
  value_counts = laptopData2022[i].value_counts()
  rows_to_keep = laptopData2022[i].isin(value_counts[value_counts > 1].index)
  laptopData2022 = laptopData2022[rows_to_keep]
#print(X.sample(5))

In [193]:
y = laptopData2022[['Price']]
X = laptopData2022.drop(columns='Price', inplace=False)

In [194]:
print(X.shape)
print(y.shape)

(1250, 12)
(1250, 1)


In [195]:
print(X.sample(5))

     Company            TypeName  Inches  Ram       OpSys  Weight  \
440   Lenovo            Notebook    17.3    8  Windows 10    2.80   
1152  Lenovo            Notebook    15.6    8       No OS    2.32   
1248  Lenovo            Notebook    15.6    8       No OS    2.30   
455     Dell  2 in 1 Convertible    11.6    4  Windows 10    1.47   
938     Dell           Ultrabook    14.0    8  Windows 10    1.36   

     Cpu_Manufacturer     Cpu_Type  Cpu_GHz Gpu_Manufacturer Memory_Type  \
440               AMD           A6      2.5              AMD         HDD   
1152            Intel       Corei5      2.3              AMD         HDD   
1248            Intel       Corei5      2.5            Intel         SSD   
455             Intel  PentiumQuad      1.6            Intel         HDD   
938             Intel       Corei5      2.5            Intel         SSD   

      Memory_Size  
440        1024.0  
1152       1024.0  
1248        256.0  
455         500.0  
938         256.0  


In [196]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

categories = [['Netbook', 'Notebook', '2 in 1 Convertible', 'Ultrabook', 'Gaming', 'Workstation']]
transformer = make_column_transformer(
    (OrdinalEncoder(categories=categories),['TypeName']),
    (OrdinalEncoder(), ['Ram', 'Memory_Size', 'Cpu_Type']),
    (KBinsDiscretizer(n_bins=5, encode='ordinal'), ['Inches', 'Weight', 'Cpu_GHz']),
    (OneHotEncoder(), ['Cpu_Manufacturer', 'Gpu_Manufacturer', 'Memory_Type', 'OpSys', 'Company']),
    remainder='passthrough'
)

pipeline = Pipeline([('preprocessing', transformer), ('model', RandomForestRegressor())]);

In [197]:
# Due to small datset size and certain unique values have to settle for overfitting as data can't be equally distributed
from sklearn.model_selection import train_test_split
print(X.shape, y.shape)
train_X, val_X, train_y, val_y =  train_test_split(X, y, test_size=0.1, random_state=1);

(1250, 12) (1250, 1)


In [198]:
from sklearn.metrics import mean_absolute_error
pipeline.fit(train_X, train_y);
train_predictions = pipeline.predict(train_X);
validation_predictions = pipeline.predict(val_X);
print('Training Error:', str(mean_absolute_error(train_predictions, train_y)))
print('Validation Error:', str(mean_absolute_error(validation_predictions, val_y)))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Training Error: 4966.01039574009
Validation Error: 9834.383850312888


In [259]:
print(laptopData2022['Memory_Size'].unique())

[ 128.  256.  512.  500. 1024.   32. 1152.   64. 1280. 2304. 2048. 1536.
  756. 2176.   16.  180. 2560.]


In [1]:
laptopManufacturer = "MSI" #@param ['Apple','HP','Acer','Asus','Dell','Lenovo','Chuwi','MSI','Microsoft','Toshiba','Huawei','Xiaomi','Vero','Razer','Mediacom','Samsung','Google','Fujitsu','LG']
laptopType = "Notebook" #@param ['Ultrabook','Notebook','Netbook','Gaming','2 in 1 Convertible','Workstation']
screenInches = 15 #@param
laptopRam = "16" #@param ['2', '4', '6', '8', '12', '16', '32']
operatingSystem = "Windows 10" #@param ['macOS','No OS','Windows 10','Mac OS X','Linux','Windows 10 S','Chrome OS','Windows 7']
weight = 1.7 #@param
cpuManufacturer = 'AMD' #@param ['Intel', 'AMD']
cpuType = 'PentiumDual' #@param ['Corei5','Corei7','A9','Corei3','CoreM','Atomx5','E','A6','CeleronDual','Ryzen1700','PentiumQuad','XeonE3','A10','A8','A12','CeleronQuad','PentiumDual','AtomX5']
cpuGHz = 1.77 #@param
gpuManufacturer = 'Nvidia' #@param ['Intel', 'AMD', 'Nvidia']
memoryType = 'SSD' #@param ['HDD', 'SSD', 'Storage']
memorySize = "1024" #@param ['128', '256', '512', '500', '1024', '32', '1152', '64', '1280', '2304', '2048', '1536', '756', '2176', '16', '180', '2560']

data = {
    'Company': [laptopManufacturer],
    'TypeName': laptopType,
    'Inches': screenInches,
    'Ram': [int(laptopRam)],
    'OpSys': [operatingSystem],
    'Weight': [float(weight)],
    'Cpu_Manufacturer': [cpuManufacturer],
    'Cpu_Type': [cpuType],
    'Cpu_GHz': [float(cpuGHz)],
    'Gpu_Manufacturer': [gpuManufacturer],
    'Memory_Type': [memoryType],
    'Memory_Size': [int(memorySize)]
}

predictionDf = pd.DataFrame(data)
print("Your model is predicted price is Rs.", pipeline.predict(predictionDf)[0]);

NameError: ignored

In [ ]:
# Project -- Best time to buy a laptop looking at historical trends (2020, 2021 i.e. discount) and predicting the expected discount given the current price for a laptop from amazon webstore in India
# E-commerce sales dataset https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data

In [ ]:
def getAllPricesFromLink(URL: str): # Get the links of webpages from amazon.in containing the required links to all sellers
  HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
  webpage = requests.get(URL, headers=HEADERS)
  soup = BeautifulSoup(webpage.content, "lxml")
  try:
    priceTag = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()
  except: return -1
  return int(priceTag)

In [ ]:
print(getAllPricesFromLink('https://www.amazon.com/Sony-PlayStation-Pro-1TB-Console-4/dp/B07K14XKZH/'))

In [ ]:
def getLinksForProduct(productName : str, productTags = []): # Get the links of webpages from amazon.in containing the required links to all sellers
  browser = uc.Chrome(); browser.get("https://www.amazon.in/");
  try: # Wait for amazon.in searchbar to load so that we can search
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#twotabsearchtextbox")));
  except: print("Unable to connect to the website, please try again ... "); return;

  searchBar = browser.find_element(By.CSS_SELECTOR, "#twotabsearchtextbox");
  searchBtn = browser.find_element(By.CSS_SELECTOR, "#nav-search-submit-button");

  searchBar.send_keys(productName);
  searchBtn.click();

  elements = browser.find_elements_by_class_name("a-size-base.a-color-secondary")
  print(len(elements))

In [ ]:
getLinksForProduct("ps4");